In [4]:
%pylab inline
#Autoreload so changing py files works
%load_ext autoreload
%autoreload 2
import gc, os, math, random

from IPython.display import Image, display, clear_output
import cv2
import time

from Util import *

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\MasterMatthew\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [7]:
class BaseFeatureExtractor:
    def __init__(self):
        pass
    def extractFeatures(image):
        #Returns a list of key points and descriptors
        pass
    
class BaseHomographyGenerator:
    def __init__(self):
        pass
    def findHomography(kp1, desc1, kp2, desc2):
        #Returns a Homography
        pass
    
class SIFTFeatureExtractor(BaseFeatureExtractor):
    def __init__(self):
        self.sift = cv2.xfeatures2d.SIFT_create()
    def extractFeatures(image):
        return sift.detectAndCompute(image, None)
    
class BFMatcherHomographyGenerator:
    def __init__(self):
        self.matcher = cv2.BFMatcher()
        pass
    def findHomography(kp1, desc1, kp2, desc2):
        matches =  matcher.knnMatch(desc1, desc2, k=2)
        good = []
        for m in matches:
            if m[0].distance < 0.5 * m[1].distance:
                good.append(m)
        matches = np.asarray(good)
        src = np.float32([ kp1[m.queryIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
        dst = np.float32([ kp2[m.trainIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
        H, masked = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
        return H

In [6]:
featureExtractor = SIFTFeatureExtractor()
homographyGenerator = BFMatcherHomographyGenerator()

In [ ]:
cap = cv2.VideoCapture(0)

def read():
    ret, frame = cap.read()
    frame = cv2.transpose(frame)
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return color, gray

#First frame should be an identity matrix
H1 = np.array([[1,0,0],[0,1,0],[0,0,1]])
#Read first frame
color, gray = read()
#Extract features from first frame
kp1, desc1 = featureExtractor.extractFeatures(gray)
#TODO: print original image

while cap.isOpened():
    #Read next frame
    color, gray = read()
    #Extract features from the next frame
    kp2, desc2 = sift.detectAndCompute(gray, None)
    #Get the homography between the previous frame and the current
    H2 = findHomography(kp1, desc1, kp2, desc2)
    #Invert homography
    H2 = np.linalg.inv(H2)
    #Multiply previous homography with current homography
    H1 = np.multiply(H1, H2)
    
    clear_output(wait=True)
    print(H1)
    
    if cv2.waitKey(32) == 32:
        break